In [ ]:
import os
import glob
import xml.etree.ElementTree as ET
import pandas as pd
import tensorflow as tf
print(tf.__version__)

2.12.0


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

# this creates a symbolic link so that now the path /content/gdrive/My\ Drive/ is equal to /mydrive
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 20230313_211046.jpg	      'H9 22-23'
'210020064 (1).pdf'	      'H9 Council Responsibilties.gdoc'
'210020064 (2).pdf'	      'HT Project G19 .gdoc'
 210020064.pdf		       Khushal_Kumawat_Manifesto_SportsCo-1.pdf
'Adobe Scan 18 Feb 2022.pdf'  'midsem paper 2022.gdoc'
'ASMP SOP.gdoc'		       photo_2022-10-04_18-36-15.jpg
'certificate (1).pdf'	       Resume_3rd_Year_draft2.pdf
 certificate.pdf	       resume.gdoc
'Colab Notebooks'	      'SOC learning.gdoc'
 CustomTF2		      'SOC Mentor Resources.gdoc'
'Fee receipt sem 4.pdf'       'SOC Work Report.gdoc'
'Finsearch Report.gdoc'       'Untitled form.gform'


In [ ]:
# clone the tensorflow models on the colab cloud vm
!git clone --q https://github.com/tensorflow/models.git

#navigate to /models/research folder to compile protos
%cd models/research

# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.

# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf2/setup.py .
!python -m pip install .

/content/gdrive/MyDrive/CustomTF2/data/models/research
Processing /content/gdrive/MyDrive/CustomTF2/data/models/research
  Preparing metadata (setup.py) ... done
  Using cached avro-python3-1.10.2.tar.gz (38 kB)
  Preparing metadata (setup.py) ... done
  Using cached apache_beam-2.49.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (14.6 MB)
  Using cached lvis-0.5.3-py3-none-any.whl (14 kB)
  Using cached tf_models_official-2.13.0-py2.py3-none-any.whl (2.6 MB)
  Using cached tensorflow_io-0.32.0-cp310-cp310-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (28.0 MB)
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
  Using cached sacrebleu-2.2.0-py3-none-any.whl (116 kB)
  Using cached portalocker-2.7.0-py2.py3-none-any.whl (15 kB)
  Using cached colorama-0.4.6-py2.py3-none-any.whl (25 kB)
  Using cached immutabledict-2.2.5-py3-none-any.whl (4.1 kB)
  Using cached PyYAML-5.4.1.tar.gz (175 kB)
  Installing build dependencies ... done
  error: subprocess-exited-with-e

In [ ]:
# testing the model builder
!python object_detection/builders/model_builder_tf2_test.py

2023-07-17 19:06:41.879463: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Traceback (most recent call last):
  File "/content/models/research/object_detection/builders/model_builder_tf2_test.py", line 24, in <module>
    from object_detection.builders import model_builder
ModuleNotFoundError: No module named 'object_detection'


In [ ]:
%cd /mydrive/CustomTF2/data/

# unzip the datasets and their contents so that they are now in /mydrive/customTF2/data/ folder
!unzip /mydrive/CustomTF2/images.zip -d .
!unzip /mydrive/CustomTF2/annotations.zip -d .

/content/gdrive/My Drive/CustomTF2/data
Archive:  /mydrive/CustomTF2/images.zip
replace ./images/0-with-mask.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: Archive:  /mydrive/CustomTF2/annotations.zip
  inflating: ./annotations/0-with-mask.xml  
  inflating: ./annotations/0.xml     
  inflating: ./annotations/1-with-mask.xml  
  inflating: ./annotations/10-with-mask.xml  
  inflating: ./annotations/100-with-mask.xml  
  inflating: ./annotations/101-with-mask.xml  
  inflating: ./annotations/103-with-mask.xml  
  inflating: ./annotations/104-with-mask.xml  
  inflating: ./annotations/105-with-mask.xml  
  inflating: ./annotations/106-with-mask.xml  
  inflating: ./annotations/107-with-mask.xml  
  inflating: ./annotations/108-with-mask.xml  
  inflating: ./annotations/109-with-mask.xml  
  inflating: ./annotations/11-with-mask.xml  
  inflating: ./annotations/110-with-mask.xml  
  inflating: ./annotations/111-with-mask.xml  
  inflating: ./annotations/112-with-mask.xml  
  inflating: ./anno

In [ ]:
#creating two dir for training and testing
!mkdir test_labels train_labels

# lists the files inside 'annotations' in a random order (not really random, by their hash value instead)
# Moves the first 274/1370 labels (20% of the labels) to the testing dir: `test_labels`
!ls annotations/* | sort -R | head -274 | xargs -I{} mv {} test_labels/


# Moves the rest of the labels ( 1096 labels ) to the training dir: `train_labels`
!ls annotations/* | xargs -I{} mv {} train_labels/

mkdir: cannot create directory ‘test_labels’: File exists
mkdir: cannot create directory ‘train_labels’: File exists
ls: cannot access 'annotations/*': No such file or directory
ls: cannot access 'annotations/*': No such file or directory


In [ ]:
#adjusted from: https://github.com/datitran/raccoon_dataset
def xml_to_csv(path):
  classes_names = []
  xml_list = []

  for xml_file in glob.glob(path + '/*.xml'):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    for member in root.findall('object'):
      classes_names.append(member[0].text)
      value = (root.find('filename').text  ,
               int(root.find('size')[0].text),
               int(root.find('size')[1].text),
               member[0].text,
               int(member[4][0].text),
               int(member[4][1].text),
               int(member[4][2].text),
               int(member[4][3].text))
      xml_list.append(value)
  column_name = ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
  xml_df = pd.DataFrame(xml_list, columns=column_name)
  classes_names = list(set(classes_names))
  classes_names.sort()
  return xml_df, classes_names

for label_path in ['train_labels', 'test_labels']:
  image_path = os.path.join(os.getcwd(), label_path)
  xml_df, classes = xml_to_csv(label_path)
  xml_df.to_csv(f'{label_path}.csv', index=None)
  print(f'Successfully converted {label_path} xml to csv.')

label_map_path = os.path.join("label_map.pbtxt")
pbtxt_content = ""

for i, class_name in enumerate(classes):
    pbtxt_content = (
        pbtxt_content
        + "item {{\n    id: {0}\n    name: '{1}'\n}}\n\n".format(i + 1, class_name)
    )
pbtxt_content = pbtxt_content.strip()
with open(label_map_path, "w") as f:
    f.write(pbtxt_content)
    print('Successfully created label_map.pbtxt ')

Successfully converted train_labels xml to csv.
Successfully converted test_labels xml to csv.
Successfully created label_map.pbtxt 


In [ ]:
#Usage:
#!python generate_tfrecord.py output.csv output_pb.txt /path/to/images output.tfrecords

#For train.record
!python /mydrive/customTF2/generate_tfrecord.py train_labels.csv  label_map.pbtxt images/ train.record

#For test.record
!python /mydrive/customTF2/generate_tfrecord.py test_labels.csv  label_map.pbtxt images/ test.record

python3: can't open file '/mydrive/customTF2/generate_tfrecord.py': [Errno 2] No such file or directory
python3: can't open file '/mydrive/customTF2/generate_tfrecord.py': [Errno 2] No such file or directory


In [ ]:
#Download the pre-trained model ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz into the data folder & unzip it.

!wget http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
!tar -xzvf ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz

--2023-07-17 19:37:08--  http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.107.128, 2607:f8b0:400e:c0d::80
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.107.128|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20515344 (20M) [application/x-tar]
Saving to: ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz.1’

ssd_mobilenet_v2_fp 100%[===================>]  19.56M  54.0MB/s    in 0.4s    

2023-07-17 19:37:09 (54.0 MB/s) - ‘ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8.tar.gz.1’ saved [20515344/20515344]

ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_mobilenet_v2_fpnlite_320x320_coco17_tpu-8/checkpoint/checkpoint
ssd_mobilenet_v2_fpnlite_320x320_coco1